In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split


In [ ]:
#Download and concatenation of data
data11 = pd.read_csv('opel_corsa_01.csv', delimiter=';')
data12 = pd.read_csv('opel_corsa_02.csv', delimiter=';')
data21 = pd.read_csv('peugeot_207_01.csv', delimiter=';')
data22 = pd.read_csv('peugeot_207_02.csv', delimiter=';')
data   = pd.concat([data11, data12, data21, data22], ignore_index=True)

data